In [3]:
from langchain_teddynote.document_loaders import HWPLoader
from transformers import AutoTokenizer
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

"""
# HWP Loader 객체 생성
loader = HWPLoader("./name.hwp")
# 문서 로드
docs = loader.load()
"""

file_paths = "./hwpfd"
def search(dirname):
    filenames = os.listdir(dirname)
    dir =[]
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        ext = os.path.splitext(full_filename)[-1]
        if ext == '.hwp': 
            dir.append(full_filename)
    return dir
file_paths=search(file_paths)
embedding_model_name = "jhgan/ko-sroberta-nli"
embeddings_model = HuggingFaceEmbeddings(
                model_name = embedding_model_name,
                model_kwargs = {"device": "cuda"},
                encode_kwargs={'normalize_embeddings':True}
    )
tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
                  tokenizer = tokenizer,
                  chunk_size = 512,
                  chunk_overlap  = 64,
                  is_separator_regex = False)
docs = []
for file_path in file_paths:
        _, file_extension = os.path.splitext(file_path)
        file_extension = file_extension.lower()
        #
        if file_extension == ".hwp":
            loader = HWPLoader(file_path)
            documents = loader.load_and_split(text_splitter=text_splitter)
        else:
            raise ValueError(f"Unsupported file type: {file_extension}")

        docs.extend(documents)



Token indices sequence length is longer than the specified maximum sequence length for this model (700 > 512). Running this sequence through the model will result in indexing errors


In [24]:
print(len(docs))
print(docs[4:8])

86
[Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='10 20 20 :2018∙ 건설기준 코드체계 전환에 따라 코드화로 통합 정비함제정(2018.6)EXCS 10 20 20 :2021∙ 코드체계화 이후 최신 건설기준을 신속히 적용하기 위해 개정함개정(2021.10)제       정  :  2018년   6월   19일심       의  :  중앙건설기술심의위원회소 관 부 서  :  국토교통부 도로정책과개       정  :  2021년  10월  19일자 문 검 토  :  국가건설기준센터 건설기준위원회관련단체 (작성기관) : 한국도로공사 (도로교통연구원)'), Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='목 차 1. 일반사항ȃ 11.1 적용 범위䏈ȃ 11.1.1 지반조사계획㩨ȃ 11.1.2 지표지질조사㩨ȃ 11.1.3 물리탐사䊐ȃ 11.1.4 시추조사䊐ȃ 11.1.5 시험굴조사와 핸드오거보링ᾬȃ 11.1.6 원위치시험과 시험굴착⟔ȃ 11.1.7 시료채취䊐ȃ 11.1.8 토질시험䊐ȃ 11.1.9 암석시험䊐ȃ 21.2 참고 기준䏈ȃ 21.2.1 지반조사계획㩨ȃ 21.2.2 지표지질조사㩨ȃ 21.2.3 물리탐사䊐ȃ 21.2.4 시추조사䊐ȃ 21.2.5 시험굴조사와 핸드오거보링ᾬȃ 21.2.6 원위치시험과 시험굴차⟔ȃ 21.2.7 시료채취䊐ȃ 21.2.8 토질시험䊐ȃ 31.2.9 암석시험䊐ȃ 31.3 용어의 정의㾴ȃ 41.4 제출물䨠ȃ 41.4.1 지반조사계획㩨ȃ 41.4.2 지표지질조사㩨ȃ 41.4.3 물리탐사䊐ȃ 41.4.4 시추조사䊐ȃ'), Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='지반조사계획㩨ȃ 41.4.2 지표지질조사㩨ȃ 41.4.3 물리탐사䊐ȃ 41.4.4 시추조사䊐ȃ

In [7]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embedding=embeddings_model)
vec_retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 5})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 5
ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vec_retriever],  
        weights=[0.7, 0.3],  
        search_type="mmr",  
    )

In [18]:
ensemble_retriever.invoke("시험굴조사에서 몇미터로 굴착해야 돼?")
# 10번 정답

[Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='명소 속성 명소 속장순재국토교통부 도로정책과김 호국토교통부 도로정책과고속도로공사 전문시방서EXCS 10 20 20 : 2021지반조사2021년 10월 발행소관부서 국토교통부관련단체 한국도로공사(39660) 경상북도 김천시 혁신8로 77 한국도로공사☎ 1588-2504(대표)http://www.ex.co.kr 작성기관 한국도로공사 도로교통연구원(18489) 경기도 화성시 동부대로 922번길 208-96☎ 031-8098-6044(품질시험센터)http://www.ex.co.kr/research국가건설기준센터(10223) 경기도 고양시 일산서구 고양대로 283(대화동)☎ 031-910-0444http://www.kcsc.re.kr'),
 Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='재측정하여야 한다.(8) 측정결과는 측선배치도, 주시곡선도, 단면도로 작성하여 정리하여야 한다.(9) 주시곡선도 또는 단면도에는 해석내용을 명시하여야 한다.3.3.3 전기비저항탐사(1) 측선의 배치는 현지의 상황에 따라 조사목적에 지장이 없는 범위 내에서 공사감독자의 승인을 받아 변경할 수 있다.(2) 전극의 배치는 탐사의 목적에 맞게 설정하고, 전극의 간격은 탐사하고자 하는 지반의 심도와 전극배열에 따라 적절히 선택할 수 있다.(3) 특히, 탐사 대상 전구간이 포함될 수 있도록 탐사측선의 연장을 충분히 계획하고, 가탐심도 및 해상도를 감안하여 적용하여야 하며, 대상구조물의 하부 지반상태를 판단할 수 있어야 한다.(4) 전극의'),
 Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='할 때 문제가 발생할 수 있는 취약구간(파쇄대, 연약대 등)에 대해서는 공사감독자

In [9]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

rerank_model = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker") #reranker모델 로드
compressor = CrossEncoderReranker(model=rerank_model, top_n=5)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=ensemble_retriever)

In [17]:
compression_retriever.invoke("시험굴조사에서 몇미터로 굴착해야 돼?")
# 2번 정답

[Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='있는 보호벽을 제거하여야 하며, 보호벽의 제거가 곤란한 경우에는 주변의 토양을 터파기한 후 지표로부터 깊이 1m 이상 보호벽을 절단하여야 한다.(9) 지하매설물 등의 지장물 예상구간은 시추작업 전 관계기관과의 협의를 통해 매설심도 및 위치를 확인하고 승인을 받은 후 시행하여야 한다.ɢ╢3.5 시험굴조사와 핸드오거보링ɢ╢3.5.1 일반(1) 땅깎기구간에서는 시험굴조사를 200m 간격으로 실시하여 흙쌓기재료의 적정성을 판단할 수 있도록 하며, 흙쌓기구간에서는 핸드오거보링을 200m 간격으로 시행하여 연약지반의 분포유무를 판단할 수 있도록 한다. 현장여건을 고려하여 조사간격은 공사감독자와 협의하여 조정할 수 있다.(2) 모든 토질시료는 통일분류방식으로 분류하며, 시험굴'),
 Document(metadata={'source': './hwpfd\\EXCS 10 20 20 지반조사.hwp'}, page_content='굴착기를 사용하여 지표면으로부터 1.2~1.5m 내외의 심도까지 굴착하여 조사한다.3.5.3 핸드오거보링(1) 흙쌓기 예정지역을 대상으로 지층 구성상태에 따른 연약지반 유무를 확인하기 위하여 실시한다.(2) 조사방법은 KS F 2319에 의하며 채취된 시료에 대한 토질분류시험을 실시한다.ɢ╢3.6 원위치시험과 시험굴착ɢ╢3.6.1 일반(1) 원위치시험이라 함은 시추조사와 병행하여 실시되는 각종 시험(표준관입시험, 공내수평재하시험, 공내투수시험, 현장투수시험, 공내전단시험, 공내영상촬영, 수압파쇄시험, 현장베인전단시험 등)과 피에조 콘 관입시험(CPT) 등을 포함한다.(2) 원위치 시험의 항목과 위치는 조사목적과 부합되는 세부 자료를 취득할 목적으로 공사감독자와 협의하여 정한다.ɢ╢(3) 시추조사공 내에서 실시되는'),
 Document(metadata={'source': './hwpfd\\EXCS 10 20 20 

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
# Prompt
template = """
다음은 건설지침을 알려주는 시방서 입니다. 
문서를 보고 질문에 참고한 문서의 내용을 있는 그대로 보여주고 다음 참고 내용 까지 보여주세요.
만약 주어진 문서 내용이 관련되지 않았다면, '문서와 관련 없습니다.'라고 답하세요.

Answer: {context}

Question: {question}

답변:
"""

from langchain_anthropic import ChatAnthropic
anthropic_api_key = "sk-"  # 실제 API 키로 대체하세요
llm = ChatAnthropic(model="claude-3-opus-20240229", anthropic_api_key=anthropic_api_key)

prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
response = rag_chain.invoke("시험굴조사에서 몇미터로 굴착해야 돼?")
print(response)

문서를 참고하면 시험굴 굴착기를 사용하여 지표면으로부터 1.2~1.5m 내외의 심도까지 굴착하여 조사한다고 되어 있습니다.
